-sandbox
<div style="text-align: center; line-height:0; padding-top: 9px;">
  <img src=https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png width="300"/>
</div>
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/databricks/koalas/master/Koalas-logo.png" width="220"/>
</div>

## De Pandas a Spark con Koalas
En esta parte del trabajo se presentará la nueva librería Koalas con la cual se cargarán los datos y se realizará un análisis exploratorio de los mismos. Se utilizarán los datos del dataset principal, así como los datos de las aerolíneas y aeropuertos de los datasets adicionales. Estos datos se pueden localizar en [Airline Delay and Cancellation Data, 2009 - 2018](https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018), [2015 Flight Delays and Cancellations](https://www.kaggle.com/usdot/flight-delays) y [US. Bureau of Transportation Statistics](https://www.bts.gov/topics/airlines-and-airports/airline-codes). 
Adicionalmente, se cargarán los datos ya limpios y pre-procesados con PySpark del año 2014 al 2017. Este conjunto de datos ha sido utilizado para el entranamiento del modelo, reservando el año 2018 como conjunto de test.

**Índice**
1. Ingesta de datos
2. Análisis Exploratorio
3. Conclusión

In [2]:
%sh
pip install koalas

Collecting koalas
 Downloading koalas-1.1.0-py3-none-any.whl (1.1 MB)
Requirement already satisfied: pandas>=0.23.2 in /databricks/python3/lib/python3.7/site-packages (from koalas) (1.0.1)
Requirement already satisfied: numpy>=1.14 in /databricks/python3/lib/python3.7/site-packages (from koalas) (1.18.1)
Requirement already satisfied: matplotlib>=3.0.0 in /databricks/python3/lib/python3.7/site-packages (from koalas) (3.1.3)
Requirement already satisfied: pyarrow>=0.10 in /databricks/python3/lib/python3.7/site-packages (from koalas) (0.15.1)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.23.2->koalas) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.23.2->koalas) (2019.3)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (0.10.0)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (2.4.6)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.7/site-packages (from matplotlib>=3.0.0->koalas) (1.1.0)
Requirement already satisfied: six>=1.0.0 in /databricks/python3/lib/python3.7/site-packages (from pyarrow>=0.10->koalas) (1.14.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.7/dist-packages (from kiwisolver>=1.0.1->matplotlib>=3.0.0->koalas) (45.2.0)
Installing collected packages: koalas
Successfully installed koalas-1.1.0
WARNING: You are using pip version 20.0.2; however, version 20.2.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

In [3]:
from pyspark import SparkContext
dbutils.library.installPyPI("koalas")
dbutils.library.restartPython()

In [4]:
import pandas as pd
import databricks.koalas as ks
import warnings
warnings.filterwarnings("ignore")
ks.set_option('compute.default_index_type', 'distributed')

# 1. Ingesta de datos

In [6]:
paths= ["/FileStore/tables/data/2014.csv", "/FileStore/tables/data/2015.csv","/FileStore/tables/data/2016.csv","/FileStore/tables/data/2017.csv"]
fulldata = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=',') \
.load(paths)
airports = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=',') \
.load('/FileStore/tables/dict/airports.csv')
airlines = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=';') \
.load('/FileStore/tables/dict/allairlines.csv')

In [7]:
fulldata.cache()
airports.cache()
airlines.cache()

Out[3]: DataFrame[CODE: string, AIRLINE: string]

## 1.1. Dataset Completo

### Pandas

In [10]:
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# # Convert the Spark DataFrame to a Pandas DataFrame using Arrow
# fullpandas_df= fulldata.select("*").toPandas()

In [11]:
pdairports = airports.toPandas()
pdairlines = airlines.toPandas()

No es posible transformar el dataset completo a Pandas por falta de memoria. En cambio si podemos cargar los datasets adicionales.

### Koalas

In [14]:
koalas_df_full = fulldata.to_koalas()
ksairports = airports.to_koalas()
ksairlines = airlines.to_koalas()

Por el contrario, con Koalas no tenemos problemas para cargar todos los datos. Esto es debido a que la arquitectura interna de Koalas es Spark.

In [16]:
koalas_df_full.cache()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2014-01-01,AA,2377,ICT,DFW,1135,1144.0,9.0,12.0,1156.0,1253.0,9.0,1300,1302.0,2.0,0.0,None,0.0,85.0,78.0,57.0,328.0,NaN,NaN,NaN,NaN,NaN,None
1,2014-01-01,AA,2378,MIA,TPA,2225,2220.0,-5.0,14.0,2234.0,2311.0,4.0,2325,2315.0,-10.0,0.0,None,0.0,60.0,55.0,37.0,204.0,NaN,NaN,NaN,NaN,NaN,None
2,2014-01-01,EV,2500,DFW,HOU,2105,NaN,NaN,NaN,NaN,NaN,NaN,2205,NaN,NaN,1.0,A,0.0,60.0,NaN,NaN,247.0,NaN,NaN,NaN,NaN,NaN,None
3,2014-01-01,EV,2502,CRW,DFW,1655,1805.0,70.0,6.0,1811.0,1941.0,24.0,1900,2005.0,65.0,0.0,None,0.0,185.0,180.0,150.0,946.0,0.0,0.0,0.0,0.0,65.0,None
4,2014-01-01,EV,2502,DFW,CRW,1320,1440.0,80.0,9.0,1449.0,1730.0,5.0,1625,1735.0,70.0,0.0,None,0.0,125.0,115.0,101.0,946.0,6.0,0.0,0.0,0.0,64.0,None
5,2014-01-01,EV,2503,AMA,DFW,1925,1909.0,-16.0,8.0,1917.0,2000.0,53.0,2030,2053.0,23.0,0.0,None,0.0,65.0,104.0,43.0,312.0,0.0,0.0,23.0,0.0,0.0,None
6,2014-01-01,EV,2503,DFW,AMA,1740,1740.0,0.0,14.0,1754.0,1848.0,5.0,1850,1853.0,3.0,0.0,None,0.0,70.0,73.0,54.0,312.0,NaN,NaN,NaN,NaN,NaN,None
7,2014-01-01,EV,2504,DFW,SHV,2040,2124.0,44.0,18.0,2142.0,2212.0,6.0,2130,2218.0,48.0,0.0,None,0.0,50.0,54.0,30.0,190.0,25.0,0.0,4.0,0.0,19.0,None
8,2014-01-01,EV,2505,AMA,DFW,1705,1733.0,28.0,9.0,1742.0,1824.0,16.0,1810,1840.0,30.0,0.0,None,0.0,65.0,67.0,42.0,312.0,0.0,0.0,2.0,0.0,28.0,None
9,2014-01-01,EV,2505,DFW,AMA,1520,1557.0,37.0,16.0,1613.0,1705.0,4.0,1630,1709.0,39.0,0.0,None,0.0,70.0,72.0,52.0,312.0,37.0,0.0,2.0,0.0,0.0,None


In [17]:
koalas_df_full.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2014-01-01,AA,2377,ICT,DFW,1135,1144.0,9.0,12.0,1156.0,1253.0,9.0,1300,1302.0,2.0,0.0,None,0.0,85.0,78.0,57.0,328.0,NaN,NaN,NaN,NaN,NaN,None
1,2014-01-01,AA,2378,MIA,TPA,2225,2220.0,-5.0,14.0,2234.0,2311.0,4.0,2325,2315.0,-10.0,0.0,None,0.0,60.0,55.0,37.0,204.0,NaN,NaN,NaN,NaN,NaN,None
2,2014-01-01,EV,2500,DFW,HOU,2105,NaN,NaN,NaN,NaN,NaN,NaN,2205,NaN,NaN,1.0,A,0.0,60.0,NaN,NaN,247.0,NaN,NaN,NaN,NaN,NaN,None
3,2014-01-01,EV,2502,CRW,DFW,1655,1805.0,70.0,6.0,1811.0,1941.0,24.0,1900,2005.0,65.0,0.0,None,0.0,185.0,180.0,150.0,946.0,0.0,0.0,0.0,0.0,65.0,None
4,2014-01-01,EV,2502,DFW,CRW,1320,1440.0,80.0,9.0,1449.0,1730.0,5.0,1625,1735.0,70.0,0.0,None,0.0,125.0,115.0,101.0,946.0,6.0,0.0,0.0,0.0,64.0,None


In [18]:
koalas_df_full.shape

Out[8]: (22931169, 28)

## 1.2. Dataset 2014

In [20]:
data = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=',') \
.load("/FileStore/tables/data/2014.csv")

In [21]:
data.cache()

Out[10]: DataFrame[FL_DATE: string, OP_CARRIER: string, OP_CARRIER_FL_NUM: int, ORIGIN: string, DEST: string, CRS_DEP_TIME: int, DEP_TIME: double, DEP_DELAY: double, TAXI_OUT: double, WHEELS_OFF: double, WHEELS_ON: double, TAXI_IN: double, CRS_ARR_TIME: int, ARR_TIME: double, ARR_DELAY: double, CANCELLED: double, CANCELLATION_CODE: string, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, DISTANCE: double, CARRIER_DELAY: double, WEATHER_DELAY: double, NAS_DELAY: double, SECURITY_DELAY: double, LATE_AIRCRAFT_DELAY: double, Unnamed: 27: string]

### Pandas

In [23]:
pandas_df= data.toPandas()

In [24]:
pandas_df.head(5)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2014-01-01,AA,2377,ICT,DFW,1135,1144.0,9.0,12.0,1156.0,1253.0,9.0,1300,1302.0,2.0,0.0,None,0.0,85.0,78.0,57.0,328.0,NaN,NaN,NaN,NaN,NaN,None
1,2014-01-01,AA,2378,MIA,TPA,2225,2220.0,-5.0,14.0,2234.0,2311.0,4.0,2325,2315.0,-10.0,0.0,None,0.0,60.0,55.0,37.0,204.0,NaN,NaN,NaN,NaN,NaN,None
2,2014-01-01,EV,2500,DFW,HOU,2105,NaN,NaN,NaN,NaN,NaN,NaN,2205,NaN,NaN,1.0,A,0.0,60.0,NaN,NaN,247.0,NaN,NaN,NaN,NaN,NaN,None
3,2014-01-01,EV,2502,CRW,DFW,1655,1805.0,70.0,6.0,1811.0,1941.0,24.0,1900,2005.0,65.0,0.0,None,0.0,185.0,180.0,150.0,946.0,0.0,0.0,0.0,0.0,65.0,None
4,2014-01-01,EV,2502,DFW,CRW,1320,1440.0,80.0,9.0,1449.0,1730.0,5.0,1625,1735.0,70.0,0.0,None,0.0,125.0,115.0,101.0,946.0,6.0,0.0,0.0,0.0,64.0,None


In [25]:
pandas_df.shape

Out[13]: (5819811, 28)

# 2. Análisis Exploratorio

In [27]:
cleandata = spark.read.format('csv').options(header='true', inferSchema='true', delimiter=',') \
.load('/my.csv')

In [28]:
koalas_df_clean = cleandata.to_koalas()

In [29]:
koalas_df_clean.shape

Out[44]: (21928586, 13)

In [30]:
koalas_df_clean.head()

,AIRLINE,ORIGIN,DESTIN,SCHED_DEPARTURE,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,DATE_TIME,AIRLINE_NAME,OR_LATITUDE,OR_LONGITUDE,DEST_LATITUDE,DEST_LONGITUDE
0,AA,ICT,DFW,1135,9.0,2.0,328.0,2014-01-01 11:35:00,American Airlines Inc.,37.64996,-97.43305,32.89595,-97.03720
1,AA,MIA,TPA,2225,-5.0,-10.0,204.0,2014-01-01 22:25:00,American Airlines Inc.,25.79325,-80.29056,27.97547,-82.53325
2,EV,CRW,DFW,1655,70.0,65.0,946.0,2014-01-01 16:55:00,ExpressJet Airlines LLC,38.37315,-81.59319,32.89595,-97.03720
3,EV,DFW,CRW,1320,80.0,70.0,946.0,2014-01-01 13:20:00,ExpressJet Airlines LLC,32.89595,-97.03720,38.37315,-81.59319
4,EV,AMA,DFW,1925,-16.0,23.0,312.0,2014-01-01 19:25:00,ExpressJet Airlines LLC,35.21937,-101.70593,32.89595,-97.03720


### Descripción

In [32]:
koalas_df_clean.describe()

,SCHED_DEPARTURE,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,OR_LATITUDE,OR_LONGITUDE,DEST_LATITUDE,DEST_LONGITUDE
count,2.192861e+07,2.192861e+07,2.192861e+07,2.192861e+07,2.190513e+07,2.190513e+07,2.190504e+07,2.190504e+07
mean,1.328892e+03,8.931490e+00,4.820624e+00,8.204281e+02,3.662859e+01,-9.597266e+01,3.666163e+01,-9.584006e+01
std,4.825177e+02,3.156505e+01,3.386701e+01,6.042187e+02,5.996387e+00,1.833328e+01,5.996623e+00,1.838458e+01
min,1.000000e+00,-8.200000e+01,-2.900000e+01,2.800000e+01,1.348345e+01,-1.766460e+02,1.348345e+01,-1.766460e+02
25%,9.200000e+02,-5.000000e+00,-1.300000e+01,3.720000e+02,3.343417e+01,-1.120081e+02,3.343417e+01,-1.119778e+02
50%,1.322000e+03,-2.000000e+00,-4.000000e+00,6.470000e+02,3.715152e+01,-9.035999e+01,3.736186e+01,-9.035999e+01
75%,1.730000e+03,8.000000e+00,8.000000e+00,1.061000e+03,4.078839e+01,-8.175517e+01,4.078839e+01,-8.159319e+01
max,2.359000e+03,3.580000e+02,2.990000e+02,4.983000e+03,7.128545e+01,-6.479856e+01,7.128545e+01,-6.479856e+01


### NaNs

In [34]:
koalas_df_clean.isna().sum()

Out[52]: AIRLINE 0
ORIGIN 0
DESTIN 0
SCHED_DEPARTURE 0
DEPARTURE_DELAY 0
ARRIVAL_DELAY 0
DISTANCE 0
DATE_TIME 0
AIRLINE_NAME 0
OR_LATITUDE 23479
OR_LONGITUDE 23479
DEST_LATITUDE 23567
DEST_LONGITUDE 23567
Name: 0, dtype: int64

In [35]:
koalas_df_clean=koalas_df_clean.dropna()
koalas_df_clean.isna().sum()

Out[55]: AIRLINE 0
ORIGIN 0
DESTIN 0
SCHED_DEPARTURE 0
DEPARTURE_DELAY 0
ARRIVAL_DELAY 0
DISTANCE 0
DATE_TIME 0
AIRLINE_NAME 0
OR_LATITUDE 0
OR_LONGITUDE 0
DEST_LATITUDE 0
DEST_LONGITUDE 0
Name: 0, dtype: int64

### Valores Totales

In [37]:
koalas_df_clean['AIRLINE'].value_counts(normalize=True)

### Visualización

In [40]:
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [41]:
#AIRLINE DECISION
#, order = koalas_df_full['OP_CARRIER'].value_counts()
num=koalas_df_full['OP_CARRIER'].to_numpy()
plt.figure(figsize=(10,7))
g=sns.countplot(x='OP_CARRIER', data=koalas_df_full, palette="deep", order = num)
g.set_xticklabels(g.get_xticklabels(),rotation=90);

--------------------------------------------------------------------------- 
 PandasNotImplementedError Traceback (most recent call last)
 <command-4091867568594568> in <module> 
 3 num = koalas_df_full [ 'OP_CARRIER' ] . to_numpy ( ) 
 4 plt . figure ( figsize = ( 10 , 7 ) ) 
 ----> 5 g = sns . countplot ( x = 'OP_CARRIER' , data = koalas_df_full , palette = "deep" , order = num ) 
 6 g . set_xticklabels ( g . get_xticklabels ( ) , rotation = 90 ) ; 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in countplot (x, y, hue, data, order, hue_order, orient, color, palette, saturation, dodge, ax, **kwargs) 
 3553 estimator , ci , n_boot , units , seed , 
 3554 orient , color , palette , saturation , 
 -> 3555 errcolor, errwidth, capsize, dodge)
 3556 
 3557 plotter . value_label = "count" 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in __init__ (self, x, y, hue, data, order, hue_order, estimator, ci, n_boot, units, seed, orient, color, palette, saturation, errcolor, errwidth, capsize, dodge) 
 1613 """Initialize the plotter.""" 
 1614 self.establish_variables(x, y, hue, data, orient,
 -> 1615 order, hue_order, units)
 1616 self . establish_colors ( color , palette , saturation ) 
 1617 self . estimate_statistic ( estimator , ci , n_boot , seed ) 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in establish_variables (self, x, y, hue, data, orient, order, hue_order, units) 
 202 # Group the numeric data 
 203 plot_data, value_label = self._group_longform(vals, groups,
 --> 204 group_names)
 205 
 206 # Now handle the hue levels for nested ordering 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in _group_longform (self, vals, grouper, order) 
 245 else : 
 246 index = None 
 --> 247 vals = pd . Series ( vals , index = index ) 
 248 
 249 # Group the val data 

 /databricks/python/lib/python3.7/site-packages/pandas/core/series.py in __init__ (self, data, index, dtype, name, copy, fastpath) 
 303 data = data . copy ( ) 
 304 else : 
 --> 305 data = sanitize_array ( data , index , dtype , copy , raise_cast_failure = True ) 
 306 
 307 data = SingleBlockManager ( data , index , fastpath = True ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/construction.py in sanitize_array (data, index, dtype, copy, raise_cast_failure) 
 447 subarr = _try_cast ( arr , dtype , copy , raise_cast_failure ) 
 448 else : 
 --> 449 subarr = _try_cast ( data , dtype , copy , raise_cast_failure ) 
 450 
 451 # scalar like, GH 

 /databricks/python/lib/python3.7/site-packages/pandas/core/construction.py in _try_cast (arr, dtype, copy, raise_cast_failure) 
 535 subarr = maybe_cast_to_integer_array ( arr , dtype ) 
 536 
 --> 537 subarr = maybe_cast_to_datetime ( arr , dtype ) 
 538 # Take care in creating object arrays (but iterators are not 
 539 # supported): 

 /databricks/python/lib/python3.7/site-packages/pandas/core/dtypes/cast.py in maybe_cast_to_datetime (value, dtype, errors) 
 1344 )
 1345 ):
 -> 1346 value = maybe_infer_to_datetimelike ( value ) 
 1347 
 1348 return value

 /databricks/python/lib/python3.7/site-packages/pandas/core/dtypes/cast.py in maybe_infer_to_datetimelike (value, convert_dates) 
 1140 if not is_list_like ( v ) : 
 1141 v = [ v ] 
 -> 1142 v = np . array ( v , copy = False ) 
 1143 
 1144 # we only care about object dtypes 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/series.py in __iter__ (self) 
 5177 
 5178 def __iter__ ( self ) : 
 -> 5179 return MissingPandasLikeSeries . __iter__ ( self ) 
 5180 
 5181 if sys . version_info >= ( 3 , 7 ) : 

 /databricks/python/lib/python3.7/site-packages/databricks/koalas/usage_logging/__init__.py in wrapper (*args, **kwargs) 
 233 def wrapper ( * args , ** kwargs ) : 
 234 try : 
 --> 235 return func ( * args , ** kwargs ) 
 236 finally : 
 237 logger . log_missing ( class_name , function_name , is_deprecated , signature ) 

 /databricks/python/lib/pytho

In [42]:
# AIRLINE DELAY AVERAGE
plt.figure(figsize=(10,7))
ax1=sns.barplot(y=koalas_df_full['OP_CARRIER'], x=koalas_df_full['ARR_DELAY'],data=koalas_df_full,palette='rocket')
plt.xlabel('ARRIVAL DELAY AVERAGE', fontsize=14)
plt.ylabel('AIRLINES', fontsize=14)
plt.show()

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-4091867568594569> in <module> 
 1 # AIRLINE DELAY AVERAGE 
 2 plt . figure ( figsize = ( 10 , 7 ) ) 
 ----> 3 ax1 = sns . barplot ( y = koalas_df_full [ 'OP_CARRIER' ] , x = koalas_df_full [ 'ARR_DELAY' ] , data = koalas_df_full , palette = 'rocket' ) 
 4 plt . xlabel ( 'ARRIVAL DELAY AVERAGE' , fontsize = 14 ) 
 5 plt . ylabel ( 'AIRLINES' , fontsize = 14 ) 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in barplot (x, y, hue, data, order, hue_order, estimator, ci, n_boot, units, seed, orient, color, palette, saturation, errcolor, errwidth, capsize, dodge, ax, **kwargs) 
 3148 estimator , ci , n_boot , units , seed , 
 3149 orient , color , palette , saturation , 
 -> 3150 errcolor, errwidth, capsize, dodge)
 3151 
 3152 if ax is None : 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in __init__ (self, x, y, hue, data, order, hue_order, estimator, ci, n_boot, units, seed, orient, color, palette, saturation, errcolor, errwidth, capsize, dodge) 
 1613 """Initialize the plotter.""" 
 1614 self.establish_variables(x, y, hue, data, orient,
 -> 1615 order, hue_order, units)
 1616 self . establish_colors ( color , palette , saturation ) 
 1617 self . estimate_statistic ( estimator , ci , n_boot , seed ) 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in establish_variables (self, x, y, hue, data, orient, order, hue_order, units) 
 153 
 154 # Figure out the plotting orientation 
 --> 155 orient = self . infer_orient ( x , y , orient ) 
 156 
 157 # Option 2a: 

 /databricks/python/lib/python3.7/site-packages/seaborn/categorical.py in infer_orient (self, x, y, orient) 
 360 elif is_not_numeric ( y ) : 
 361 if is_not_numeric ( x ) : 
 --> 362 raise ValueError ( no_numeric ) 
 363 else : 
 364 return "h" 

 ValueError : Neither the `x` nor `y` variable appears to be numeric.

In [43]:
# CHECKING OUTLIERS
colors = ['firebrick', 'gold', 'lightcoral', 'aquamarine', 'c', 'yellowgreen', 'grey',
          'seagreen', 'tomato', 'violet', 'wheat', 'chartreuse', 'lightskyblue', 'royalblue']
plt.figure(figsize=(10,7))
ax2=sns.stripplot(y="AIRLINE_NAME", x="ARRIVAL_DELAY", size = 4, palette = colors,
                    data=list_data[0], linewidth = 0.5,  jitter=True)
plt.xlabel('ARRIVAL DELAY OUTLIERS', fontsize=14)
plt.ylabel('AIRLINES', fontsize=14)
plt.figure(figsize=(10,7))
plt.show()